In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

plt.style.use("ggplot")


# Setup

In [ ]:
data = Path("data")
raw = data / "raw"
processed = data / "processed"

csvs = [*data.rglob("*.csv")]

if not raw.exists() or not processed.exists():
    raw.mkdir(exist_ok=True)
    processed.mkdir(exist_ok=True)
    for csv in csvs:
        shutil.move(csv, raw)


# Chargement des données

On charge les données depuis le zip téléchargé depuis Kaggle

In [ ]:
customers = pd.read_csv(raw / "olist_customers_dataset.csv")
customers.head()


In [ ]:
geolocations = pd.read_csv(raw / "olist_geolocation_dataset.csv")
geolocations.head()


In [ ]:
order_items = pd.read_csv(raw / "olist_order_items_dataset.csv")
order_items.head()


In [ ]:
order_payments = pd.read_csv(raw / "olist_order_payments_dataset.csv")
order_payments.head()


In [ ]:
order_reviews = pd.read_csv(raw / "olist_order_reviews_dataset.csv")
order_reviews.head()


In [ ]:
orders = pd.read_csv(raw / "olist_orders_dataset.csv")
orders.head()


In [ ]:
products = pd.read_csv(raw / "olist_products_dataset.csv")
products.head()


In [ ]:
sellers = pd.read_csv(raw / "olist_sellers_dataset.csv")
sellers.head()


On n'a pas besoin du fichier `.csv` "product_category_name_translation"

In [ ]:
customers.describe(include="all")


In [ ]:
geolocations.describe(include="all")


In [ ]:
order_payments.describe(include="all")


In [ ]:
order_items.describe(include="all")


In [ ]:
order_reviews.describe(include="all")


In [ ]:
orders.describe(include="all")


In [ ]:
products.describe(include="all")


Un produit qui a un poids de 0g peut exister. Mais un produit qui fait 0g et qui a une taille de 7cm je ne sais pas... Peut être qu'on doit vérifier quelque chose à ce niveau là.

In [ ]:
products[(products.product_weight_g == 0) | products.product_weight_g.isna()]


Je ne veux pas supprimer ces données car je n'arrive pas très bien à savoir si elles sont exactes ou non. Il semblerait que la taille de ces produits ai été saisie avec des tailles par défaut. Mais impossible d'en être sûr.

De plus si je retire les lignes de la table, on aura certainement une erreur si un client a commandé un de ces produits. Retenons donc qu'il y a une possible erreur dans cette table.

In [ ]:
# Erreur de nommage des colonnes. C'est "length" et non "lenght"
products.columns = products.columns.map(lambda x: x.replace("lenght", "length"))
products.columns


In [ ]:
sellers.describe(include="all")


# Traitements

In [ ]:
customers.isna().sum()


In [ ]:
geolocations.isna().sum()


In [ ]:
order_payments.isna().sum()


In [ ]:
order_items.isna().sum()


In [ ]:
order_reviews.isna().sum()


In [ ]:
orders.isna().sum()


In [ ]:
products.isna().sum()


In [ ]:
sellers.isna().sum()


Pas beaucoup de valeurs nulls dans ce dataset. On voit bien qu'un tri a été effectué.

On voit qu'il y a eu des reviews pour lesquelles il n'y a pas de titre ou de messages associés. Cette information est possiblement intéressante. On pourrait même faire un feature engineering pour "client ayant pris le temps de laisser un message". Car évidemment on ne va pas analyser les messages de ceux-ci.

On peut donner un niveau d'avis à ces personnes : 
- 0 = Pas de titre ou message
- 1 = Titre
- 2 = Titre et message

In [ ]:
# Il y a des valeurs avec " " comme valeur
order_reviews.loc[
    order_reviews.review_comment_title == " ", "review_comment_title"
] = None


In [ ]:
order_reviews["review_level"] = order_reviews.review_comment_title.isna().replace(
    {True: 0, False: 1}
) + order_reviews.review_comment_message.isna().replace({True: 0, False: 1})
order_reviews["review_level"].value_counts()


In [ ]:
order_reviews = order_reviews.drop(
    columns=["review_comment_title", "review_comment_message"]
)
order_reviews.head()


On peut en profiter pour effectuer un autre feature engineering sur le différence entre la date de création de la review et sa réponse

In [ ]:
from datetime import datetime

date_fmt = "%Y-%m-%d %H:%M:%S"
to_datetime = lambda x: datetime.strptime(x, date_fmt)

creation = order_reviews.review_creation_date.map(to_datetime)
answer = order_reviews.review_answer_timestamp.map(to_datetime)
delay = answer - creation
# Conservons le nombre de minutes passées avant une réponse
minutes_f = lambda x: x.total_seconds() / 60
order_reviews["review_answer_delay"] = delay.map(minutes_f)
# On peut supprimer les autres dates car elle ne sont plus utiles pour nous. On aurait pu les utiliser pour remarquer une potentielle amélioration du traitement des retours mais ce n'est pas l'objectif ici
order_reviews = order_reviews.drop(
    columns=["review_creation_date", "review_answer_timestamp"]
)
order_reviews.head()


Parmis les valeurs null. On remarque qu'il y a également certaines dates pour les commandes qui ne sont pas définies

In [ ]:
orders[orders.order_approved_at.isna()]


J'étais curieux de savoir pourquoi certaines commandes n'avaient pas été approuvées. La raison est que les commandes ont été annulées. Je vais quand même vérifier qu'il n'y a que des commandes annulées

In [ ]:
orders[orders.order_approved_at.isna()].order_status.unique()


C'est embêtant...

In [ ]:
orders[orders.order_approved_at.isna() & (orders.order_status != "canceled")]


Je ne vais pas perdre du temps à essayer de comprendre pourquoi car je ne trouve pas cette feature si intéressante que cela pour identifier des clusters. Mais ça aurait été intéressant de savoir comment une commande peut être livrée sans être approuvée.

In [ ]:
# Je profite de drop la colonne approved pour également drop celle correspondant à l'arrivée chez l'expéditeur car elle n'apportera pas de plus value selon moi
orders = orders.drop(columns=["order_approved_at", "order_delivered_carrier_date"])
orders.head()


On peut s'intéresser au délai de livraison d'une commande. C'est, je pense, la mesure la plus appropriée à obtenir pour identifier des clusters. Mais on a des valeurs nulls qu'on va devoir traiter avant.

In [ ]:
orders.order_status.value_counts()


In [ ]:
never_delivered = orders.order_delivered_customer_date.isna()
orders.loc[never_delivered].order_status.value_counts()


Pour plus de simplicité je propose de ne pas prendre en compte ces valeurs car il y a sûrement eu un problème avec la commande. On remarque que les valeurs pour lesquelles il n'y a pas de date de livraison sont celles qui ont un statut différent de "delivered" (à quelques exceptions près. Certaines commandes ont été annulées mais ont été livrées).

Pour les quelques livraisons marquées comme livrées mais avec aucun date, on pourra utiliser la date estimée.

Calculer une différence de date à partir des autres données est compliqué. On peut donc ignorer ces commandes.

In [ ]:
is_delivered = orders.order_status.isin(["delivered"])

orders.loc[
    never_delivered & is_delivered, "order_delivered_customer_date"
] = orders.loc[never_delivered & is_delivered, "order_estimated_delivery_date"]

orders[never_delivered].head()


In [ ]:
orders_with_dates = orders.loc[is_delivered]
purchase = orders_with_dates.order_purchase_timestamp.map(to_datetime)
estimated = orders_with_dates.order_estimated_delivery_date.map(to_datetime)
delivered = orders_with_dates.order_delivered_customer_date.map(to_datetime)

orders.loc[is_delivered, "estimation_error"] = (delivered - estimated).map(minutes_f)
orders.loc[is_delivered, "delivery_delay"] = (delivered - purchase).map(minutes_f)
orders.head()


In [ ]:
orders.describe()


In [ ]:
orders.estimation_error.plot.hist()


On a des outliers. Dans ce cas c'est des commandes qui sont arrivées à destination mais très longtemps après.

Ce ne sera peut être pas pratique pour faire des clusters du fait des valeurs nulls mais sait-on jamais. Ca pourrait être important au moins pour notre analyse.

Si on se base sur un avis client, il n'est jamais vraiment content quand sa commande dépasse le délai prévu par l'entreprise. Ainsi ça peut être important pour avoir un avis. On peut même refaire un feature engineering pour savoir si ce délai est dépassé ou non. Ce qui nous permettra de colorier les commandes qui ne se sont pas bien passées.

In [ ]:
orders.loc[:, "delay_respected"] = orders.estimation_error <= 0
orders.delay_respected.value_counts()


# Analyse

- Sur Kaggle, avec les graphiques par défauts, on arrive à voir que la majorité des commandes se sont produites en 2018 avec 1 en 2016 et quelques unes en 2017

Avec la façon dont la BD est réalisée, on peut juste utiliser la table de client pour connaître le nombre de commandes d'un client. Mais on va quand même effectuer un merge avec la table orders.

In [ ]:
customer_orders = pd.merge(customers, orders, on="customer_id")
customer_orders.head()


In [ ]:
number_of_orders = customer_orders.customer_unique_id.value_counts()
number_of_orders.head()


In [ ]:
number_of_orders.value_counts(normalize=True).sort_index()


In [ ]:
order_count_ids = customer_orders.customer_unique_id.value_counts()
(order_count_ids > 1).sum() / order_count_ids.count()


On a 3% de clients qui ont passé plus d'1 commande.

In [ ]:
orders.order_status.value_counts(normalize=True).plot.bar()
print(orders.order_status.value_counts(normalize=True)["delivered"])


97% des commandes sont livrées. Analysons un peu l'argent qui a été dépensé par les clients.

In [ ]:
orders_payments = pd.merge(orders, order_payments, on="order_id")
orders_payments


In [ ]:
print(
    f"customer_length={len(customer_orders)}, order_payments_length={len(order_payments)}"
)


On a une relation de 1 à plusieurs entre les 2 tables. Un client peut payer plusieurs fois et d'après les informations de la table, de différentes façons.

In [ ]:
customers_orders_payments = pd.merge(customers, orders_payments, on="customer_id")
customers_orders_payments.head()


In [ ]:
customers_orders_payments.payment_value.sort_values().reset_index(drop=True).plot.line()


In [ ]:
customers_orders_payments.count()


In [ ]:
customers_payment_values = (
    customers_orders_payments.groupby("customer_unique_id")
    .payment_value.agg(["sum", "mean", min, max])
    .reset_index()
)
customers_payment_values.head()


In [ ]:
customers_payment_values["sum"].sort_values().reset_index(drop=True).plot.line()
customers_payment_values["mean"].sort_values().reset_index(drop=True).plot.line()
customers_payment_values["max"].sort_values().reset_index(drop=True).plot.line()
customers_payment_values["min"].sort_values().reset_index(drop=True).plot.line()


On remarque bien un creux en bas à droite qui nous indique qu'il y a certains très bons clients qui peuvent dépenser beaucoup d'argent.

Ce graphique est quasi le même que celui avec la distribution des commandes (sans regrouper par client) car justement la majorité des clients passent 1 seule commande.

Pas de variance visible entre `mean`, `min`, `max`, `sum` (encore car la majorité ont 1 seule commande)

In [ ]:
best_customers_orders_payments = customers_orders_payments.customer_unique_id.isin(
    number_of_orders[(number_of_orders > 1)].index
)


In [ ]:
best_customers_payment_values = (
    customers_orders_payments.loc[best_customers_orders_payments]
    .groupby("customer_unique_id")
    .payment_value.agg(["sum", "mean", min, max])
    .reset_index()
)
best_customers_payment_values["sum"].sort_values().reset_index(drop=True).plot.line()
best_customers_payment_values["mean"].sort_values().reset_index(drop=True).plot.line()
best_customers_payment_values["max"].sort_values().reset_index(drop=True).plot.line()
best_customers_payment_values["min"].sort_values().reset_index(drop=True).plot.line()
plt.legend()


Avec les clients ayant passés plus d'une commande, on voit mieux l'écart que je souhaitais voir avant. Surtout au niveau du creux en bas à droite.

Attention, une valeur `x` ne correspond pas à un client !

On anayse ici uniquement la croissance de la `sum`, `mean`, `min`, `max`.

Regardons les catégories de produit :

In [ ]:
category_names = products.product_category_name.value_counts(normalize=True)
others = category_names < 0.02
sum_of_others = category_names[others].sum()
category_names = category_names[~others]
category_names["other"] = sum_of_others


In [ ]:
plt.figure(figsize=(16, 6))
plt.title("Product category distribution trucated by others")
plt.pie(category_names, labels=category_names.index, autopct="%1.1f%%")
plt.show()


## RFM

### Récence

In [ ]:
order_dates = orders.order_purchase_timestamp.apply(to_datetime)
max_order_date = max(order_dates)
max_order_date


In [ ]:
order_date_diff = (max_order_date - order_dates).map(minutes_f)
order_date_diff


In [ ]:
orders["order_date_lately"] = order_date_diff


### Fréquence

On a déjà calculé le nombre de fois qu'un client a passé une commande avec la variable `number_of_orders`. 

Ce qu'on peut faire est donc de calculer la fréquence réelle des commandes faites par un client. On souhaite donc avoir un nombre correspondant au délai entre 2 commandes. Et ensuite on peut prendre la moyenne de ces délais. On peut considérer que s'il n'y a qu'une seule commande (comme c'est le cas le plus souvent), alors le délai est de 0.

Cela ne devrait pas être un problème pour le clustering avec certains algorithmes qui pourront reconnaître des groupes entre nos données malgré le fait qu'avec un calcul de distance euclidienne ils soient éloignés. C'est le principe de t-SNE par exemple.

In [ ]:
def calculate_diffs(o):
    o = o.order_purchase_timestamp.values
    if(len(o) == 1):
        return 0
    time_diffs = pd.Series([])
    for i in range(len(o) - 1):
        o1 = to_datetime(o[i + 1])
        o2 = to_datetime(o[i])
        time_diffs[len(time_diffs)] = minutes_f(o1 - o2)

    return time_diffs.mean()

In [ ]:
# On va donc ordoner d'abord nos dates de commandes par client
F = customer_orders.sort_values(by="order_purchase_timestamp").groupby(by="customer_unique_id").apply(calculate_diffs)
F.head()

### Montant

On a déjà les montants nécessaires pour notre clustering

Le plus utile pour moi est le lien entre client/commandes/paiements

# Export

Je propose donc d'exporter nos données vers un format plus facilement utilisable pour notre clustering

In [ ]:
big_group = pd.merge(customers, orders, on="customer_id")
big_group = big_group.loc[big_group.order_delivered_customer_date.notna() & big_group.delivery_delay.notna()]

big_group = pd.merge(big_group, order_payments, on="order_id")

Certains clients peuvent habiter dans des quartiers un peu plus chic. On peut donc se dire que la position est importante. Reportons les données de localisation dans notre export

In [ ]:
geolocations = geolocations.groupby(by="geolocation_zip_code_prefix")[["geolocation_lat", "geolocation_lng"]].mean()
geolocations.head()

In [ ]:
mean = geolocations[["geolocation_lat", "geolocation_lng"]].mean()
mean

In [ ]:
big_group[["lat", "lng"]] = big_group.customer_zip_code_prefix.apply(lambda x: geolocations.loc[x] if (geolocations.index == x).any() else mean)
big_group.head()

In [ ]:
big_group.to_csv(processed / "customers.csv", index=False)

## Client unique

On peut directement faire une version avec les clients groupés

In [ ]:
grouped_customer = big_group.groupby("customer_unique_id")
unique_customer = grouped_customer[["order_date_lately","delivery_delay", "estimation_error"]].mean()
unique_customer = unique_customer.rename(columns={'order_date_lately': 'recency'})
unique_customer.head()

In [ ]:
unique_customer["number_of_orders"] = grouped_customer.customer_unique_id.count()
unique_customer.head()

Pour le délai respecté, on peut discrétiser la variable en mettant un ratio du nombre de fois qu'il a été respecté

In [ ]:
unique_customer["respected_ratio"] = (grouped_customer.delay_respected.sum() / unique_customer.number_of_orders)
unique_customer.head()

On va ajouter les dépenses

In [ ]:
unique_customer["amount"] = grouped_customer.payment_value.sum()
unique_customer.head()

In [ ]:
unique_customer["respected_ratio"]

In [ ]:
big_group.head()

In [ ]:
# Un client qui a passé plusieurs commande peut avoir plusieurs lat/lng différentes. On peut prendre la position spaciale la plus fréquente, cela représentera son "domicile"
unique_customer[["lat", "lng"]] = grouped_customer[["lat", "lng"]].agg(pd.Series.mode)

On va maintenant intégrer la fréquence de commande par client qui est dans la variable `F`

In [ ]:
unique_customer.shape

In [ ]:
F.shape

C'est normal qu'on ai plus de données dans F car on avait supprimé les clients avec les commandes où la livraison était NaN

In [ ]:
unique_customer["frequency"] = unique_customer.index.map(lambda x: F[x])
unique_customer.head()

In [ ]:
unique_customer.to_csv(processed / "unique_customer_orders.csv", index=False)